In [0]:
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt

In [0]:
# Titanic Datasets - Survived or Not
 
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
gender_submission_data = pd.read_csv("gender_submission.csv")

In [54]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [55]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [56]:
train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [60]:
# how many survived
train_data['Survived'].value_counts() # 549-survived 342-died

0    549
1    342
Name: Survived, dtype: int64

In [61]:
  train_data['Pclass'].value_counts() # base on passenger class 1st class - 216 people, 2nd class - 184-people, 3rd class - 491-people

3    491
1    216
2    184
Name: Pclass, dtype: int64

In [63]:
train_data['Sex'].value_counts() # Male count - 577 and Female Count - 314

male      577
female    314
Name: Sex, dtype: int64

In [65]:
train_data['Embarked'].value_counts() # Embark - Onboard 

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [0]:
from sklearn.base import BaseEstimator, TransformerMixin

class DataFrameSelector(BaseEstimator, TransformerMixin):
  def __init__ (self, attrib):
    self.attrib = attrib
  def fit(self, X, y=None):
    return self
  def transform(self,X):
    return X[self.attrib]

In [0]:
# build the pipeline for numerical attributes
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

num_pipeline = Pipeline([
                         ("num_attrib",DataFrameSelector(['Age','SibSp','Parch','Fare'])),
                         ('imputer', SimpleImputer(strategy='median'))
])

In [77]:
num_pipeline.fit_transform(train_data)

array([[22.    ,  1.    ,  0.    ,  7.25  ],
       [38.    ,  1.    ,  0.    , 71.2833],
       [26.    ,  0.    ,  0.    ,  7.925 ],
       ...,
       [28.    ,  1.    ,  2.    , 23.45  ],
       [26.    ,  0.    ,  0.    , 30.    ],
       [32.    ,  0.    ,  0.    ,  7.75  ]])

In [0]:
class MostFrequentImmputer(BaseEstimator, TransformerMixin):
  def fit(self, X, y=None):
    self.most_frequent = pd.Series([X[c].value_counts().index[0] for c in X], index=X.columns)
    return self
  def transform(self, X, y=None):
    return X.fillna(self.most_frequent)

In [0]:
from sklearn.preprocessing import OneHotEncoder

In [0]:
cat_pipeline = Pipeline([
                         ('cat_attr',DataFrameSelector(['Pclass','Sex','Embarked'])),
                         ('imputer', MostFrequentImmputer()),
                         ('Encoder',OneHotEncoder(sparse=False)),
])

In [90]:
cat_pipeline.fit_transform(train_data)

array([[0., 0., 1., ..., 0., 0., 1.],
       [1., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 1.],
       ...,
       [0., 0., 1., ..., 0., 0., 1.],
       [1., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 1., 0.]])

In [0]:
from sklearn.pipeline import FeatureUnion
preprocess_pipeline = FeatureUnion(transformer_list=[
                                                     ('num_pipeline', num_pipeline),
                                                     ('cat_pipeline', cat_pipeline),
])

In [93]:
X_train = preprocess_pipeline.fit_transform(train_data)
X_train

array([[22.,  1.,  0., ...,  0.,  0.,  1.],
       [38.,  1.,  0., ...,  1.,  0.,  0.],
       [26.,  0.,  0., ...,  0.,  0.,  1.],
       ...,
       [28.,  1.,  2., ...,  0.,  0.,  1.],
       [26.,  0.,  0., ...,  1.,  0.,  0.],
       [32.,  0.,  0., ...,  0.,  1.,  0.]])

In [0]:
y_train = train_data['Survived']

In [98]:
# Start the prediction using SVC
from sklearn.svm import SVC
svm_clf = SVC(gamma="auto")
svm_clf.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [0]:
X_test = preprocess_pipeline.fit_transform(test_data)
y_pred = svm_clf.predict(X_test)

In [132]:
from sklearn.model_selection import cross_val_score
svm_scores = cross_val_score(svm_clf,X_train,y_train,cv=5)
svm_scores.mean()

0.7194965789969242

In [135]:
# RandomForestClassifiers
from sklearn.ensemble import RandomForestClassifier
for_clf = RandomForestClassifier(n_estimators=50,random_state=42)
for_scores = cross_val_score(for_clf,X_train,y_train,cv=5)
for_scores.mean()

0.8036407005209968

In [0]:
for_train = for_clf.fit(X_train,y_train)

In [144]:
for_test = for_clf.predict(X_test)
from sklearn.metrics import mean_squared_error

final_mse = mean_squared_error(y_test, for_test)
final_rmse = np.sqrt(final_mse)
final_rmse

0.4207534822469881

In [0]:
# Random Forest model is giving good result compare to SVC model 
# let try another step to increase our model accuracy base on hyperparameter tuning using grid search cv

from sklearn.model_selection import GridSearchCV

# creat param grid for random forst classifier
param_grid = {
    'max_depth':[30,40,50],
    'max_features':[2,3,4,5],
    'min_samples_split':[8,10,12,14,16],
    'n_estimators':[100,200,300,400]
}

for_cls = RandomForestClassifier()

grid_search = GridSearchCV(estimator=for_cls,param_grid=param_grid,cv=3,n_jobs=1,verbose=2)


In [0]:
grid_search.fit(X_train,y_train)

In [127]:
grid_search.best_estimator_

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=40, max_features=5,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=8,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [129]:
np.sqrt(grid_search.cv_results_['mean_test_score'])

array([0.90391343, 0.90515421, 0.90391343, 0.90329239, 0.90824877,
       0.90577397, 0.90886641, 0.90701221, 0.90515421, 0.91010045,
       0.90701221, 0.9063933 , 0.90204905, 0.9063933 , 0.90515421,
       0.90453403, 0.90204905, 0.90577397, 0.90329239, 0.90204905,
       0.90515421, 0.9076307 , 0.90948364, 0.9076307 , 0.9076307 ,
       0.90886641, 0.91071684, 0.91256351, 0.90948364, 0.91194837,
       0.91256351, 0.91071684, 0.9076307 , 0.9076307 , 0.9063933 ,
       0.9063933 , 0.9076307 , 0.90391343, 0.9063933 , 0.90824877,
       0.91317824, 0.91194837, 0.91071684, 0.91071684, 0.90701221,
       0.91379256, 0.91194837, 0.91133281, 0.91010045, 0.91071684,
       0.91071684, 0.91133281, 0.90948364, 0.90515421, 0.90701221,
       0.90824877, 0.90267093, 0.90886641, 0.90948364, 0.90701221,
       0.90886641, 0.90577397, 0.90824877, 0.91071684, 0.90886641,
       0.91133281, 0.9076307 , 0.91194837, 0.9063933 , 0.90886641,
       0.9063933 , 0.90824877, 0.90886641, 0.90824877, 0.90886

In [0]:
final_model = grid_search.best_estimator_
final_pred = final_model.predict(X_test)

In [138]:
from sklearn.metrics import mean_squared_error

final_mse = mean_squared_error(y_test, final_pred)
final_rmse = np.sqrt(final_mse)
final_rmse

0.3594253787238923

SUMMARY:

Model is little bit improving after hyperparamater tuning
mean squared error is less compared to previous one 
we still need to improve the model 